# Model

1. Dividir el set de datos entre valores dependientes (y) e independientes (X). Y sería la columna
que se quiere predecir (“churn”).
2. Generar sets de testeo y entrenamiento.
3. ¿Es necesario escalar las features? Hacerlo si fuera necesario.
4. Probar por lo menos dos modelos y seleccionar uno. Explicar porque lo selecciono y qué
métricas uso para decidir.

<hr/>

In [82]:
# Libraries

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
plt.close("all")

import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold


In [83]:
df_churn = pd.read_csv('../data/preprocessed/churn_preprocessed.csv')

## Primera selección de features

Aquí se trabajará con dos modalidades de selección de features, **'con género'** y **'sin género'**.
<br/>
Ambas se evaluarán de la misma manera y la selección del modelo quedará a disposición del cliente.



In [103]:
df_churn = df_churn.drop(['Unnamed: 0'], axis=1)

In [104]:
df_churn

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,No,Yes,No,1.0,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,No,No,No,34.0,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,Male,No,No,No,2.0,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,No,No,No,45.0,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,No,No,No,2.0,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6027,Male,No,Yes,Yes,24.0,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.50,No
6028,Female,No,Yes,Yes,72.0,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.90,No
6029,Female,No,Yes,Yes,11.0,No,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
6030,Male,No,Yes,No,4.0,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.60,Yes


In [106]:
#

X = df_churn.drop('Churn', axis=1)
y = df_churn.Churn

X = pd.get_dummies(X, drop_first=True)
y = pd.get_dummies(y, drop_first=True)

In [107]:
X.sample(5)

,tenure,MonthlyCharges,TotalCharges,gender_Male,Partner_Yes,Dependents_Yes,PhoneService_Yes,MultipleLines_No phone service,MultipleLines_Yes,InternetService_Fiber optic,...,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
5975,30.0,94.10,2804.45,0,1,0,1,0,1,1,...,0,1,0,0,0,0,1,1,0,0
2988,54.0,101.50,5373.10,0,1,0,1,0,1,1,...,0,1,0,1,1,0,1,1,0,0
2306,1.0,78.45,78.45,0,0,0,1,0,0,1,...,0,1,0,0,0,0,0,0,1,0
2600,34.0,49.80,1734.20,1,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,1,0
1592,21.0,58.85,1215.45,0,1,1,1,0,0,0,...,0,1,0,0,1,0,1,1,0,0


<hr/>

### No correr

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X,y, stratify=y, random_state=12)

Churn = Yes shows same proportion of the dataframe before and after split x, y, train and test. It means that stratify= y is keeping proportions through dataframe splitting process.

In [18]:
df_churn.Churn.value_counts()['Yes'] / df_churn.shape[0]

0.26492042440318303

In [19]:
y_train.value_counts()['Yes'] / y_train.shape[0]

0.26480990274093724

In [20]:
y_test.value_counts()['Yes'] / y_test.shape[0]

0.26525198938992045

Aplicamos dummies sobre variables categóricas.

In [21]:
X_train = pd.get_dummies(X_train, drop_first=True)
X_test = pd.get_dummies(X_test, drop_first=True)
X_train.sample(5)

,tenure,MonthlyCharges,TotalCharges,gender_Male,SeniorCitizen_Yes,Partner_Yes,Dependents_Yes,PhoneService_Yes,MultipleLines_No phone service,MultipleLines_Yes,...,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
1807,39.0,19.75,757.95,0,0,1,1,1,0,0,...,1,0,1,0,0,1,0,0,0,1
4675,58.0,105.50,6205.50,0,1,1,0,1,0,0,...,0,1,0,1,1,0,1,1,0,0
4951,62.0,84.45,4959.15,0,1,0,0,1,0,1,...,0,1,0,0,1,0,0,0,1,0
4727,1.0,20.05,20.05,1,0,0,0,1,0,0,...,1,0,1,0,0,0,0,0,0,1
3446,10.0,98.50,1037.75,1,0,1,0,1,0,1,...,0,1,0,1,0,0,1,0,1,0


## Modelo

Un caso típico de clasigicación es la Regresión Logística.
El primer conjunto de modelos trabajará con este algoritmo.

Trabajaremos con GridSearch para seleccionar el mejor modelo en los casos de CON GÉNERO y SIN GÉNERO.
* La métrica de principal será F1.
* Se mostrarán matrices de confusión.
* Los parámetros de la grilla serán:
    * penalty
    * ...

In [124]:
params_grid = [
    {'penalty': ['l1','l2','elasticnet', 'none'],
     'C': np.logspace(-4,4,20),
     'solver': ['saga'],
     'max_iter': [100,1000]
     }
]
f1 = make_scorer(f1_score , average='macro')


### Modelo con género

In [146]:
modelo_1 = LogisticRegression()

In [147]:
modelo_1_grid = GridSearchCV(
    modelo_1, 
    param_grid= params_grid, 
    cv = 3,
    n_jobs=-1,
    scoring=f1
    )

In [148]:
modelo_1_fit = modelo_1_grid.fit(X,y)

/home/gera/miniconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/gera/miniconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/gera/miniconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/gera/miniconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please chan

In [149]:
modelo_1_fit.best_estimator_

LogisticRegression(C=0.0001, max_iter=1000, penalty='none', solver='saga')

In [150]:
print(f'Accuracy: {modelo_1_fit.score(X,y): .3f}')

Accuracy:  0.705


<hr/>

## Selección de features

Debido a la gran cantidad de features en relación con el tamaño del dataset, puede ser interesante reducir la cantidad de features para mejorar la performance del modelo.

In [136]:
X_1 = df_churn[['SeniorCitizen','tenure','PaymentMethod', 'MonthlyCharges']]
X_1 = pd.get_dummies(X, drop_first=True)

In [133]:
modelo_2 = LogisticRegression()

In [140]:
modelo_2_grid = GridSearchCV(
    modelo_1, 
    param_grid= params_grid, 
    cv = 3,
    n_jobs=-1,
    scoring=f1
    )

In [141]:
modelo_2_fit = modelo_2_grid.fit(X_1,y)

/home/gera/miniconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/gera/miniconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/gera/miniconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/gera/miniconda3/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warning

In [142]:
modelo_2_fit.best_estimator_

LogisticRegression(C=0.0001, max_iter=1000, penalty='none', solver='saga')

In [143]:
print(f'Accuracy: {modelo_2_fit.score(X,y): .3f}')

Accuracy:  0.705


<hr/>

## Modelo: Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
random_forest_1 = RandomForestRegressor(n_estimators=1000, 
                                      criterion='mse', 
                                      max_depth = 4, 
                                      bootstrap=True, 
                                      n_jobs = -1, 
                                      random_state = 127,
                                      max_samples= 0.3)

In [ ]:
X_2 = df_churn.drop('Churn', axis=1)
y_2 = df_churn['Churn']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, stratify=y, random_state=12)

<hr/>

In [ ]:
recall_score(y_test,y_test_pred, pos_label='No')
precision_score(y_test, y_test_pred_1, pos_label='No')

print('Specificity=', (TN)/ (TN+FP))
print('Total de casos negativos predichos correctamente (TN) =',(TN))
print('Total de casos negativos (TN+FP) =',(TN+FP))

f1_score(y_test,y_test_pred_1, pos_label='No')

<hr/>

In [75]:
y_test_pred_1 = modelo_1.predict(X_test)
y_test_pred_1

array(['No', 'Yes', 'No', ..., 'No', 'No', 'Yes'], dtype=object)

El método .predict_proba() devuelve un array con dos probabilidades para cada instancia del test set: 
p(y=0) y p(y=1), en ese orden.

La primera columna es la probabilidad de pertenecer a la clase 0 (negativa), y la segunda columna a la clase 1 (positiva).

In [76]:
y_test_pred_proba = model_1.predict_proba(X_test)
y_test_pred_proba

array([[0.97435831, 0.02564169],
       [0.41691179, 0.58308821],
       [0.6105662 , 0.3894338 ],
       ...,
       [0.98495995, 0.01504005],
       [0.66787784, 0.33212216],
       [0.20134248, 0.79865752]])

### Métricas generales

In [77]:
accuracy_score(y_test, y_test_pred_1)

0.7931034482758621

In [78]:
model_1.intercept_

array([-0.26223816])

In [79]:
model_1.coef_

array([[-5.88935272e-02,  6.79101730e-03,  2.42911969e-04,
        -3.07966559e-02,  1.71577093e-01, -5.59663480e-04,
        -1.38845379e-01, -6.19481208e-01,  3.57243052e-01,
         2.87780114e-01,  7.81634638e-01, -6.91957234e-02,
        -6.91957234e-02, -4.68684075e-01, -6.91957234e-02,
        -2.39237856e-01, -6.91957234e-02, -1.22103470e-01,
        -6.91957234e-02, -4.07764121e-01, -6.91957234e-02,
         1.51987866e-01, -6.91957234e-02,  1.70724201e-01,
        -5.23366555e-01, -8.34416408e-01,  3.34498620e-01,
        -1.26254540e-01,  2.79533876e-01,  1.85891691e-03]])

Matriz de confusión

In [80]:
confusion = confusion_matrix(y_test, y_test_pred_1)

TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]

In [81]:
print('TP: ', TP,' TN: ', TN,' FP: ',FP,' FN: ',FN )
print('Accuracy=', (TP+TN)/ (TP+TN+FP+FN))
print('Total de casos correctamente predichos (TP+TN) =',(TP+TN))
print('Total de casos (TP+TN+FP+FN) =',(TP+TN+FP+FN))

TP:  209  TN:  987  FP:  121  FN:  191
Accuracy= 0.7931034482758621
Total de casos correctamente predichos (TP+TN) = 1196
Total de casos (TP+TN+FP+FN) = 1508


### Métricas específicas

**RECALL:** Si su valor es bajo, es porque hay presencia de falsos negativos. Por eso, esta medida es sensible a los FN.

Comparado con accuracy_score, esta medida se enfoca en los casos positivos, así muestra cómo funciona nuestro modelo en relación al objeto de interés de nuestro negocio.

Útil cuando la ocurrencia de falsos negativos es inaceptables.

In [82]:
recall_score(y_test,y_test_pred, pos_label='No')

0.8907942238267148

**PRECISION:** Si su valor es bajo, es porque hay presencia de falsos positivos. Por eso, esta medida es sensible a los FP.

Util cuando necesitamos estar seguros de los verdaderos positivos.

In [83]:
precision_score(y_test, y_test_pred_1, pos_label='No')

0.8378607809847198

**Specificity:** (especificidad o true negative rate (TNR)) es la proporción de negativos correctamente predichos sobre el total de casos negativos.

Mide qué tan "específico" es el clasificador al predecir las instancias positivas. Se calcula como el número de verdaderos negativos (TN) sobre todos los casos que son negativos (TN+FP).

Si su valor es bajo, es porque hay presencia de falsos positivos. Por eso, esta medida es sensible a los FP.

Otro ejemplo donde importa una alta especificidad, es si predecimos que una persona está enferma al cual debemos suministrarle una droga potente, y no lo está realmente.

In [84]:
print('Specificity=', (TN)/ (TN+FP))
print('Total de casos negativos predichos correctamente (TN) =',(TN))
print('Total de casos negativos (TN+FP) =',(TN+FP))

Specificity= 0.8907942238267148
Total de casos negativos predichos correctamente (TN) = 987
Total de casos negativos (TN+FP) = 1108


**F1-Score:** Como regla general, cuanto mayor es esta métrica, mejor es el modelo.

Pero para tener un f1-score alto, es necesario que tanto recall como precision sean altos, mientras que un f1-score bajo puede ser el resultado de un valor bajo en por lo menos una de estas métricas o en ambas a la vez.

In [85]:
f1_score(y_test,y_test_pred_1, pos_label='No')

0.8635170603674541

La ventaja de usar la media armónica (en vez de la media aritmética) es que el resultado del f1-score no es sensible a valores altos de una de las dos variables (recall o precision).

Por otro lado, no todos los valores extremos son ignorados, ya que los que son muy bajos si tienen peso en el resultado final.

<hr/>

### IMPORTANTE

* Debemos tener en cuenta Recall cuando no podemos aceptar los falsos negativos.
* Specificity cuando no debemos aceptar falsos positivos.
* Precision cuando debemos estar seguros de los verdaderos positivos.

**Cuál es la pregunta de negocio? Qué métrica se quiere maximizar?**
En base a esto decidiremos cuales son las mejoras a aplicar en los siguientes pasos.

<hr/>

In [ ]:
model_2 = LogisticRegression(penalty='L1')
model_2.fit(X_train, y_train);

In [86]:
y_test_pred_2 = model_1.predict(X_test)
y_test_pred_2

array(['No', 'Yes', 'No', ..., 'No', 'No', 'Yes'], dtype=object)

In [87]:
f1_score(y_test,y_test_pred_2, pos_label='No')

0.8635170603674541